In [8]:
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
import sys
sys.path.append("..")

In [2]:
qlen = 10
klen = 10
context_position = torch.arange(qlen, dtype=torch.long,
                                        )[:, None]
memory_position = torch.arange(klen, dtype=torch.long,
                                      )[None, :]

In [6]:
memory_position - context_position

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [-1,  0,  1,  2,  3,  4,  5,  6,  7,  8],
        [-2, -1,  0,  1,  2,  3,  4,  5,  6,  7],
        [-3, -2, -1,  0,  1,  2,  3,  4,  5,  6],
        [-4, -3, -2, -1,  0,  1,  2,  3,  4,  5],
        [-5, -4, -3, -2, -1,  0,  1,  2,  3,  4],
        [-6, -5, -4, -3, -2, -1,  0,  1,  2,  3],
        [-7, -6, -5, -4, -3, -2, -1,  0,  1,  2],
        [-8, -7, -6, -5, -4, -3, -2, -1,  0,  1],
        [-9, -8, -7, -6, -5, -4, -3, -2, -1,  0]])

In [11]:
from src.model.layers.relative_position_bias import RelativePositionBias

pos_embed = RelativePositionBias(bidirectional=False,
                                 num_buckets=20,
                                 max_distance=10,
                                 n_heads=8)

In [15]:
pos_embed._relative_position_bucket(memory_position - context_position, bidirectional=True,
                                    num_buckets=19, max_distance=10)

tensor([[ 0, 10, 11, 12, 13, 14, 15, 16, 16, 17],
        [ 1,  0, 10, 11, 12, 13, 14, 15, 16, 16],
        [ 2,  1,  0, 10, 11, 12, 13, 14, 15, 16],
        [ 3,  2,  1,  0, 10, 11, 12, 13, 14, 15],
        [ 4,  3,  2,  1,  0, 10, 11, 12, 13, 14],
        [ 5,  4,  3,  2,  1,  0, 10, 11, 12, 13],
        [ 6,  5,  4,  3,  2,  1,  0, 10, 11, 12],
        [ 7,  6,  5,  4,  3,  2,  1,  0, 10, 11],
        [ 7,  7,  6,  5,  4,  3,  2,  1,  0, 10],
        [ 8,  7,  7,  6,  5,  4,  3,  2,  1,  0]])

In [2]:
import torch

# 创建一个三维张量，例如，大小为[2, 4, 4]
x = torch.randn(2, 4, 4)

# 在第二个维度（channels）上应用torch.max()
values, indices = torch.max(x, dim=1)
print(x)
print("Max values:", values)
print("Indices of max values:", indices)

tensor([[[-0.3324, -1.4082, -0.0784,  0.9798],
         [-1.5731,  2.0689,  0.8974,  1.5193],
         [-0.4568,  0.1283,  0.5386, -2.4981],
         [-1.6869,  0.1335,  0.8823, -0.2645]],

        [[-0.6141,  0.9644, -0.2441, -1.4504],
         [ 0.7161, -0.5827,  0.6977,  1.3610],
         [ 0.8721, -0.4602,  1.0663,  0.3069],
         [-1.1508, -0.7910,  1.0085, -0.4246]]])
Max values: tensor([[-0.3324,  2.0689,  0.8974,  1.5193],
        [ 0.8721,  0.9644,  1.0663,  1.3610]])
Indices of max values: tensor([[0, 1, 1, 1],
        [2, 0, 2, 1]])
